# MRJOB

## Co se naučíte?

* seznámíte se s knihovnou MRJOB
* vyzkoušíte ladící režim (nevyžaduje Hadoop)
* spustíte MRJOB v dokerizovaném Hadoopu

[MRJob](https://mrjob.readthedocs.io/en/latest/index.html#) je knihovna pro Python, která nám umožňuje snadno psát úlohy typu *Map* a *Reduce*. Její výhodou je poměrně slušná dokumentace a hlavně zde odpadá nutnost mít nainstalovaný *Hadoop*, protože je možné ji spouštět i lokálně. Z mého pohledu jako plus hodnotím i to, že je použit objektový přístup a uživatel si pouze podědí od bázové třídy a doplní příslušné metody.

**Poznámka**

Pro otestování uvedených příkladů stačí překopírovat buňky do pythonovského souboru a pak normálně spouštět z příkazové řádky.

## Map Reduce - Hello World

Téměř ve všech materiálech věnovaných Map a Reduce (*MR*), je ukázáno počítání počtu slov v nějakém dokumentu. Pojďme si na této úloze ilustrovat danou knihovnu *MRJob*.

Na ukázce níže je zmiňovaná implementace počítání počtu slov. Nejprve je vytvořena třída, která dědí od *MRJob*, pak už stačí jen redefinovat metody *mapper* a *reducer*. Metoda *mapper* slouží k tomu, že přijímá textová data a provádí na nich transformaci tak, že získáme dvojici *klíč* a *hodnota*. V "našem" příkladu je vždy vytvořena dvojice slovo a číslo jedna. Metodou *reduce* právě provádíme operace nad hodnotami příslušejícími daným klíčům. Tedy *reducer* dostává ke každému klíči **generátor** hodnot. V níže uvedeném příkladu jsou pak sečteny všechny hodnoty, tedy všechny jedničky a dostáváme počet výskytů daného slova. V tomto minimalistickém příkladu bylo vynechána metoda *combiner*, která slouží ke slučování výsledků přicházejících z *mapperu*. 

Poslední částí je pak zavolání metody *run* pro spuštění.

In [ ]:
from mrjob.job import MRJob

class WordCounter(MRJob):
	def mapper(self, _, line):
		for word in line.split():
			yield(word, 1)
	def reducer(self, word, counts):
		yield(word, sum(counts))

if __name__ == '__main__':
	WordCounter.run()

V dalším předpokládám, že program se jmenuje *wc.py* a spustíme jej v terminálu příkazem, kde *data.txt* je soubor, který chceme zpracovávat. Výstup bude produkován na standardní výstup *STDOUT*.

```
python3 wc.py -r inline data.txt

```

**Poznámka**

Parametry příkazové řádky nám umožňují spouštět MRJob ve více módech. Pro nás nejzajímavější jsou *inline*, který je spouštěn v jednom procesu a je určen pro testování, zároveň je defaultní volbou, více viz [zde](https://mrjob.readthedocs.io/en/latest/runners-inline.html). Druhá volba je *local*, zde je již využito spuštění na více procesech, více viz [zde](https://mrjob.readthedocs.io/en/latest/runners-local.html).

## Map Reduce - Četnosti slov

Nyní si ukážeme mírně komplikovanější úlohu. Program určí *n* slov s největší četností a pak nakreslí jejich historgram. Nejprve se podívejme na určování četností. Pro určení četností je nutné udělat více průchodů, tedy více operací map a reduce. Každý průchod map, combine, reduce se nazývá *step* (stupeň), viz [MRStep](https://mrjob.readthedocs.io/en/latest/step.html?highlight=mrstep) a výstupy jednotlivých stupňů jsou takto skládány. Napojení těchto stupňů se odehrává v metodě *steps*.

Na ukázce níže jsou použity dva stupně, kde první stupeň používá všechny tři metody map, combine, reduce a druhý jen reduce. V mapperu prvního stupně (*mapper_map_words*) jsou jednotlivé řádky parsovány pomocí regulárních výrazů, viz [zde](https://docs.python.org/3/library/re.html), a každému slovu je přiřazena jednička. Jednotlivých instancí mapperů může obecně být více a každý bude zpracovávat část úlohy, proto se obvykle nad nimi píše metoda combine (*combiner*), která výsledky skládá a posílá dále. Vede to často ke zmenšení objemu přenášených dat. V naší ukázce v kombineru sčítáme počet výskytů nad daným slovem. První step je ukončen operací reduce, kde jsou opět sčítány výskyty slov, tedy výsledky z combine a následně je vytvořen generátor, který **nevrací klíč**, viz *None* na dané pozici a jako příslušná hodnota je použita dvojice slovo a četnost.

**Poznámka**

Lze to udělat i tak, že budeme vracet dvojici slovo a počet výskytů. Jako **samostatný úkol** upravte daný kód. Pozor, bude nutné upravit i reducer ve druhém stupni.

Druhý stupeň je tvořen samostatným reducerem, který už jen výsledky setřídí a propustí deset nejčetnějších slov.



In [ ]:
# import knihoven a nastaveni
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

rex = re.compile(r"\b[\w']{2,}") # vytvoreni regularniho vyrazu, chci slova delsi nez dva znaky
nth = 10 # kolik nejcastejsich slov

In [ ]:
# vlastni implementace úlohy
class Count(MRJob):
    def mapper_map_words(self,_, line):
        for word in rex.findall(line):
                yield (word.lower(), 1)
    def combiner_combine_words_count(self, word, counts):
        yield (word, sum(counts))
    def reducer_count_words(self, word, counts):
        yield None, (sum(counts), word)
    def reducer_max_word(self,_,count_word_pair):
        for count, word in sorted(count_word_pair, reverse=True)[:nth]:
            yield (word, count)
    def steps(self):
        return [
            MRStep(mapper=self.mapper_map_words,
                   combiner=self.combiner_combine_words_count,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_max_word)
        ]

if __name__ =='__main__':
    Count.run()


Pro otestování jsme si stáhněte například román [RUR](https://www.gutenberg.org/ebooks/59112) a uložili jej jako *rur.txt*. Za předpokladu, že program se jmenuje *wordhistogram.py* jej spustíme následujícím kódem. Všimněte si přesměrování výsledků do souboru (*>*) *wordresults.txt*.
```
python3 wordhistogram.py -r local rur.txt > wordresults.txt
```
Dále je již jen vykreslení výsledků z daného souboru.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

if __name__=="__main__":
    data = pd.read_csv("wordresults.txt", sep="\t", header=None, index_col=0)
    print(data.head())
    ax = data.plot.bar(legend='')
    plt.xticks(rotation = 45)
    plt.title("Cetnosti slov")
    plt.show()


## Map Reduce - Průměrná teplota 

Nyní si ukážeme mírně komplikovanější úlohu, kde budeme číst ze vstupního *csv* souboru údaje o počasí, budeme počítat dení průměry a určíme počet dní v měsíci, kdy byla teplota vyšší než daná hodnota. Předpokládáme data v csv formátu, viz tento [soubor](https://1url.cz/@jf_meteo), který si ovšem musíte rozbalit.

Výše zmíněná komplikace spočívá v tom, že nepočítáme průměry za celý den, ale jen pro nějaké pevně stanovené časy v 7, 14 a 21 hodin. 
Další komplikace spočívá v tom, že operací map může být spušteno více a není možné počítat průměr přímo v kombineru nad touto operací a pak ještě v reduceru, neboť by to počítalo průměr průměrů.

V kódu níže je jedno z možných řešení. Opět jsou použity dva stupně. V prvním stupni se pracuje na úrovni dnů. Nejprve dojde, v operaci map (*mapper_map_day_temperature*) k naparsování info o počasí a filtrování hodnot na příslušné časy. Nakonec je vytvořen generátor, kde klíč je pouze datum (den) a hodnota je teplota ve stanovenou hodnotu. V následném combineru (*combiner_combine_temp*) je pro každý den vracena dvojice součet teplot a počet sečtených hodnot. Reducer prvního stupně (*reducer_filter_temperature*) opět přijímá tyto údaje a už počítá celkový průměr teploty v daném dni. Pokud je teplota vyšší než zadaná teplota (*temperature_filter*) přiřadí k danému dni hodnotu jedna, v opačném případě nula, čímž dochází k filtrování hodnot.

Druhý stupeň pouze počítá počet dnů průměrnou teplotou vyšší než je zadaná teplota. Je to analogické určení počtu slov. Jediné co je třeba udělat je převádět klíče z formátu dnů na měsíce.


In [ ]:
from mrjob.job import MRJob
from mrjob.step import MRStep
from _datetime import datetime

header_line = "datetime"
datetime_index = 0
outtemp_index = 3

selected_hours = set([7, 14, 21])
temperature_filter = 20.0


class Count(MRJob):

    def mapper_map_day_temperature(self, _, line):
        items = line.split(',')
        if items[datetime_index] != 'datetime':  # preskoc prvni radek
            date = datetime.strptime(items[datetime_index], '%Y-%m-%d %H:%M:%S')
            if date.hour in selected_hours and date.minute == 0:
                yield (f"{date.date()}", float(items[outtemp_index]))  # date.date() float(items[2])

    def combiner_combine_temp(self, day, temperatures):
        suma = 0
        counter = 0
        for value in temperatures:
            suma += value
            counter += 1
        yield (day, (suma, counter))

    def reducer_filter_temperature(self, day, val):
        c = 0
        total = 0
        for s, n in val:
            c += n
            total += s
        # zda ma prumernou teplotu - za vsechna mereni
        if total/c >= temperature_filter:
            yield  (day, 1) #(day, f"{total / c}-->{1}")
        else:
            yield (day, 0) #(day, f"{total / c}-->{0}")

    def mapper_map_day_to_month(self, day, val):
        yield (day[:7], val)

    def combiner_sum_days(self, month, val):
        yield (month, sum(val))

    def reducer_sum_days(self, month, val):
        yield (month, sum(val))

    def steps(self):
        return [
            MRStep(mapper=self.mapper_map_day_temperature,
                   combiner=self.combiner_combine_temp,
                   reducer=self.reducer_filter_temperature),
            MRStep(mapper=self.mapper_map_day_to_month,
                   combiner=self.combiner_sum_days,
                   reducer=self.reducer_sum_days)

        ]


if __name__ == '__main__':
    Count.run()


Za předpodkladu, že stažený soubor se jmenuje *meteo.csv* a daný skript *weather.py* spustíme z příkazové řádky následovně:

```
 python3 weather.py -r local meteo.csv

```



## Distribuované spuštění

Pro distribuované prostředí je potřeba mít nakonfigurováno prostředí pro Hadoop streaming (tj. skript musí být spustitelný na všech uzlech) včetně dostupnosti `mrjob` a vstupní data musí být v HDFS.

Vlastní spuštění je pak už snadné.

`python3 weather.py -r hadoop hdfs://user/root/meteo.csv`

> **Úkoly**: Vyzkoušejte distribované zpracování a porovnejte jeho efektivitu s lokálním řešením (zohledněte to, že v případě využití docker verze je zpracování de facto stále lokální)